# The Ergodic Insurance Advantage

## Overview
- **What this notebook does:** Demonstrates the fundamental insight of ergodic economics applied to insurance -- while insurance appears expensive from an ensemble (expected value) perspective, it becomes optimal when viewed through time-average growth rates.
- **Prerequisites:** [getting-started/02_quick_start.ipynb](../getting-started/02_quick_start.ipynb), [core/01_loss_distributions.ipynb](01_loss_distributions.ipynb)
- **Estimated runtime:** 2--5 minutes (depends on `N_SCENARIOS`)
- **Audience:** [Executive] / [Practitioner]

## Key Concepts
- **Ensemble Average:** Expected value across many parallel scenarios.
- **Time Average:** Growth rate experienced by a single entity over time.
- **Ergodic Theory:** For multiplicative processes (like wealth), these two averages diverge.
- **Insurance Puzzle:** Why rational actors buy "expensive" insurance -- resolved by time-average thinking.

In [ ]:
"""Google Colab setup: mount Drive and install package dependencies.

Run this cell first. If prompted to restart the runtime, do so, then re-run all cells.
This cell is a no-op when running locally.
"""
import sys, os
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')

    NOTEBOOK_DIR = '/content/drive/My Drive/Colab Notebooks/ei_notebooks/core'

    os.chdir(NOTEBOOK_DIR)
    if NOTEBOOK_DIR not in sys.path:
        sys.path.append(NOTEBOOK_DIR)

    !pip install ergodic-insurance -q 2>&1 | tail -3
    print('\nSetup complete. If you see numpy/scipy import errors below,')
    print('restart the runtime (Runtime > Restart runtime) and re-run all cells.')

## Setup

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from ergodic_insurance import Config, ManufacturerConfig
from ergodic_insurance.manufacturer import WidgetManufacturer
from ergodic_insurance.loss_distributions import ManufacturingLossGenerator
from ergodic_insurance.insurance import InsurancePolicy, InsuranceLayer
from ergodic_insurance.simulation import Simulation, SimulationResults
from ergodic_insurance.ergodic_analyzer import ErgodicAnalyzer

plt.style.use('seaborn-v0_8-darkgrid')

# Reproducibility
np.random.seed(42)

## Configuration

Widget Manufacturing Inc. -- a $10M manufacturer used as the running example throughout this framework.

In [ ]:
# Company parameters
INITIAL_ASSETS = 10_000_000
ASSET_TURNOVER = 1.2
BASE_OPERATING_MARGIN = 0.10
TAX_RATE = 0.25
RETENTION_RATIO = 0.70

# Simulation parameters
TIME_HORIZON = 50
N_SCENARIOS = 50

# Loss parameters (sustainable for demonstration)
LOSS_FREQUENCY = 0.10
LOSS_SEVERITY_MEAN = 300_000
LOSS_SEVERITY_STD = 400_000

manufacturer_config = ManufacturerConfig(
    initial_assets=INITIAL_ASSETS,
    asset_turnover_ratio=ASSET_TURNOVER,
    base_operating_margin=BASE_OPERATING_MARGIN,
    tax_rate=TAX_RATE,
    retention_ratio=RETENTION_RATIO,
)
base_manufacturer = WidgetManufacturer(manufacturer_config)

claim_generator = ManufacturingLossGenerator.create_simple(
    seed=42, frequency=LOSS_FREQUENCY,
    severity_mean=LOSS_SEVERITY_MEAN, severity_std=LOSS_SEVERITY_STD,
)

expected_annual_loss = LOSS_FREQUENCY * LOSS_SEVERITY_MEAN
ebit = INITIAL_ASSETS * ASSET_TURNOVER * BASE_OPERATING_MARGIN

print(f"Widget Manufacturing Inc.")
print(f"  Initial assets: ${INITIAL_ASSETS:,.0f}")
print(f"  Annual revenue: ${INITIAL_ASSETS * ASSET_TURNOVER:,.0f}")
print(f"  EBIT: ${ebit:,.0f}")
print(f"  Expected annual loss: ${expected_annual_loss:,.0f}")
print(f"  Losses as % of EBIT: {expected_annual_loss / ebit * 100:.1f}%")
print(f"  EBIT / expected losses: {ebit / expected_annual_loss:.1f}x")

## 1. Insurance Policy Design

Design a two-layer insurance policy. The premium exceeds expected losses -- the traditional view labels this as "expensive."

In [ ]:
insurance_layers = [
    InsuranceLayer(attachment_point=50_000, limit=1_500_000, rate=0.02),
    InsuranceLayer(attachment_point=1_550_000, limit=3_500_000, rate=0.008),
]

insurance_policy = InsurancePolicy(layers=insurance_layers, deductible=50_000)
total_premium = sum(layer.limit * layer.rate for layer in insurance_layers)
premium_to_loss = total_premium / expected_annual_loss if expected_annual_loss > 0 else 0

print(f"Insurance Structure:")
print(f"  Primary: ${insurance_layers[0].attachment_point:,.0f}--"
      f"${insurance_layers[0].attachment_point + insurance_layers[0].limit:,.0f} "
      f"at {insurance_layers[0].rate * 100:.1f}%")
print(f"  Excess:  ${insurance_layers[1].attachment_point:,.0f}--"
      f"${insurance_layers[1].attachment_point + insurance_layers[1].limit:,.0f} "
      f"at {insurance_layers[1].rate * 100:.1f}%")
print(f"  Annual premium: ${total_premium:,.0f}")
print(f"  Premium / Expected Loss: {premium_to_loss:.2f}x")
print(f"  Premium as % of EBIT: {total_premium / ebit * 100:.1f}%")

## 2. Run Simulations: Insured vs. Uninsured

We run parallel Monte Carlo simulations comparing insured and uninsured outcomes.

In [ ]:
def run_simulation_batch(n_scenarios, insurance=None, seed_offset=0):
    """Run batch of simulations with or without insurance."""
    results = []
    for i in range(n_scenarios):
        manufacturer = base_manufacturer.copy()
        claim_gen = ManufacturingLossGenerator.create_simple(
            seed=42 + seed_offset + i, frequency=LOSS_FREQUENCY,
            severity_mean=LOSS_SEVERITY_MEAN, severity_std=LOSS_SEVERITY_STD,
        )
        sim = Simulation(
            manufacturer=manufacturer, loss_generator=claim_gen,
            time_horizon=TIME_HORIZON, insurance_policy=insurance,
            seed=42 + seed_offset + i,
        )
        results.append(sim.run())
        if (i + 1) % 10 == 0:
            print(f"  Completed {i + 1}/{n_scenarios}")
    return results

print(f"Running {N_SCENARIOS} INSURED scenarios over {TIME_HORIZON} years...")
insured_results = run_simulation_batch(N_SCENARIOS, insurance=insurance_policy, seed_offset=0)

print(f"\nRunning {N_SCENARIOS} UNINSURED scenarios over {TIME_HORIZON} years...")
uninsured_results = run_simulation_batch(N_SCENARIOS, insurance=None, seed_offset=1000)

print(f"\nTotal simulation-years computed: {N_SCENARIOS * 2 * TIME_HORIZON:,}")

## 3. Ergodic Analysis

Compare ensemble-average and time-average growth rates.

In [ ]:
analyzer = ErgodicAnalyzer()
comparison = analyzer.compare_scenarios(
    insured_results=insured_results,
    uninsured_results=uninsured_results,
    metric="equity",
)

print("ERGODIC ANALYSIS RESULTS")
print("=" * 60)

print("\nENSEMBLE AVERAGE (Expected Value):")
print(f"  Insured:   {comparison['insured']['ensemble_average'] * 100:.2f}% per year")
print(f"  Uninsured: {comparison['uninsured']['ensemble_average'] * 100:.2f}% per year")

ins_ta = comparison['insured']['time_average_median']
unins_ta = comparison['uninsured']['time_average_median']

print("\nTIME AVERAGE (Individual Experience):")
if np.isfinite(ins_ta) and np.isfinite(unins_ta):
    print(f"  Insured:   {ins_ta * 100:.2f}% per year (median)")
    print(f"  Uninsured: {unins_ta * 100:.2f}% per year (median)")
    print(f"  Difference: {(ins_ta - unins_ta) * 100:.2f}%")
else:
    print("  Insufficient surviving scenarios for reliable estimates.")

print("\nSURVIVAL RATES:")
print(f"  Insured:   {comparison['insured']['survival_rate'] * 100:.1f}%")
print(f"  Uninsured: {comparison['uninsured']['survival_rate'] * 100:.1f}%")
print(f"  Survival gain: {comparison['ergodic_advantage']['survival_gain'] * 100:.1f}%")

## 4. Visualize Growth Paths

Compare sample growth trajectories, final wealth distributions, growth rate distributions, and survival curves.

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Sample paths
ax = axes[0, 0]
n_paths = 20
for i in range(min(n_paths, len(insured_results))):
    if len(insured_results[i].equity) > 0 and insured_results[i].equity[-1] > 0:
        ax.plot(insured_results[i].years, insured_results[i].equity, alpha=0.3, color='blue', lw=0.5)
for i in range(min(n_paths, len(uninsured_results))):
    if len(uninsured_results[i].equity) > 0 and uninsured_results[i].equity[-1] > 0:
        ax.plot(uninsured_results[i].years, uninsured_results[i].equity, alpha=0.3, color='red', lw=0.5)
ax.set_xlabel('Years'); ax.set_ylabel('Equity ($)'); ax.set_yscale('log')
ax.set_title('Sample Growth Paths (Blue=Insured, Red=Uninsured)'); ax.grid(True, alpha=0.3)

# Final wealth distribution
ax = axes[0, 1]
insured_final = [r.equity[-1] for r in insured_results if len(r.equity) > 0 and r.equity[-1] > 0]
uninsured_final = [r.equity[-1] for r in uninsured_results if len(r.equity) > 0 and r.equity[-1] > 0]
if insured_final or uninsured_final:
    bins = np.logspace(5, 9, 30)
    if insured_final:
        ax.hist(insured_final, bins=bins, alpha=0.5, label='Insured', color='blue', density=True)
    if uninsured_final:
        ax.hist(uninsured_final, bins=bins, alpha=0.5, label='Uninsured', color='red', density=True)
    ax.set_xscale('log'); ax.legend()
ax.set_xlabel('Final Equity ($)'); ax.set_ylabel('Density')
ax.set_title('Final Wealth Distribution'); ax.grid(True, alpha=0.3)

# Growth rate distribution
ax = axes[1, 0]
ins_growth = [g for g in (analyzer.calculate_time_average_growth(r.equity) for r in insured_results
              if len(r.equity) > 0) if np.isfinite(g)]
unins_growth = [g for g in (analyzer.calculate_time_average_growth(r.equity) for r in uninsured_results
                if len(r.equity) > 0) if np.isfinite(g)]
if ins_growth:
    ax.hist(ins_growth, bins=30, alpha=0.5, label='Insured', color='blue', density=True)
    ax.axvline(np.median(ins_growth), color='blue', ls='--', label=f'Median: {np.median(ins_growth)*100:.1f}%')
if unins_growth:
    ax.hist(unins_growth, bins=30, alpha=0.5, label='Uninsured', color='red', density=True)
    ax.axvline(np.median(unins_growth), color='red', ls='--', label=f'Median: {np.median(unins_growth)*100:.1f}%')
ax.set_xlabel('Time-Average Growth Rate'); ax.set_ylabel('Density')
ax.set_title('Growth Rate Distribution'); ax.legend(); ax.grid(True, alpha=0.3)

# Survival curves
ax = axes[1, 1]
max_years = max(
    max((len(r.years) for r in insured_results), default=0),
    max((len(r.years) for r in uninsured_results), default=0),
)
if max_years > 0:
    years = np.arange(max_years)
    ins_surv = [sum(1 for r in insured_results if len(r.equity) > t and r.equity[t] > 0)
               / len(insured_results) for t in range(max_years)]
    unins_surv = [sum(1 for r in uninsured_results if len(r.equity) > t and r.equity[t] > 0)
                 / len(uninsured_results) for t in range(max_years)]
    ax.plot(years, np.array(ins_surv) * 100, label='Insured', color='blue', lw=2)
    ax.plot(years, np.array(unins_surv) * 100, label='Uninsured', color='red', lw=2)
    ax.legend(); ax.set_ylim([0, 105])
ax.set_xlabel('Years'); ax.set_ylabel('Survival Rate (%)')
ax.set_title('Survival Rates Over Time'); ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. The Insurance Puzzle Resolution

This section crystallizes the core insight of the framework.

In [ ]:
print("THE INSURANCE PUZZLE RESOLUTION")
print("=" * 60)

print("\n1. TRADITIONAL VIEW (Ensemble Average):")
print(f"   Expected annual loss: ${expected_annual_loss:,.0f}")
print(f"   Annual premium paid:  ${total_premium:,.0f}")
print(f"   Premium/Loss ratio:   {premium_to_loss:.2f}x")
if premium_to_loss > 1:
    print("   --> Insurance appears EXPENSIVE (premium > expected loss)")

print("\n2. ERGODIC VIEW (Time Average):")
if np.isfinite(ins_ta) and np.isfinite(unins_ta):
    print(f"   Growth WITH insurance:    {ins_ta * 100:.2f}% per year")
    print(f"   Growth WITHOUT insurance: {unins_ta * 100:.2f}% per year")
    growth_gain = ins_ta - unins_ta
    print(f"   Growth improvement:       {growth_gain * 100:.2f}% per year")
    if growth_gain > 0:
        years_ex = 20
        ins_mult = (1 + ins_ta) ** years_ex
        unins_mult = (1 + unins_ta) ** years_ex
        print(f"\n   After {years_ex} years:")
        print(f"   Insured wealth:   {ins_mult:.1f}x initial")
        print(f"   Uninsured wealth: {unins_mult:.1f}x initial")
        print(f"   Advantage: {(ins_mult / unins_mult - 1) * 100:.1f}% more wealth")

print("\n3. WHY THE DIFFERENCE?")
print("   Wealth growth is MULTIPLICATIVE, not additive.")
print("   A single catastrophic loss permanently impairs compounding.")
print("   Insurance converts unpredictable large losses into predictable small costs.")
print("   This volatility reduction enhances geometric (time-average) growth.")

print("\n4. THE RESOLUTION:")
print("   Rational actors maximize time-average growth, not expected value.")
print("   Insurance is not a cost center -- it is an investment in growth stability.")

## 6. Summary Statistics

In [ ]:
summary_data = {
    'Metric': [
        'Ensemble Average Growth', 'Time Average Growth (Median)',
        'Survival Rate', 'Annual Premium', 'Premium/Loss Ratio',
    ],
    'Insured': [
        f"{comparison['insured']['ensemble_average'] * 100:.2f}%",
        f"{comparison['insured']['time_average_median'] * 100:.2f}%",
        f"{comparison['insured']['survival_rate'] * 100:.1f}%",
        f"${total_premium:,.0f}",
        f"{premium_to_loss:.2f}x",
    ],
    'Uninsured': [
        f"{comparison['uninsured']['ensemble_average'] * 100:.2f}%",
        f"{comparison['uninsured']['time_average_median'] * 100:.2f}%",
        f"{comparison['uninsured']['survival_rate'] * 100:.1f}%",
        "$0", "N/A",
    ],
}

print(pd.DataFrame(summary_data).to_string(index=False))

## Key Takeaways

- **Ensemble average overstates uninsured returns.** It averages across many parallel worlds; you only live in one.
- **Time average captures individual experience.** Volatility drag from large losses permanently impairs compounding.
- **Insurance enhances growth.** Despite premiums exceeding expected losses by 50--200%, the time-average growth rate is higher with insurance.
- **Survival is non-negotiable.** Insurance dramatically improves survival probability, which is a prerequisite for long-term compounding.
- **The "premium puzzle" is resolved.** People buy insurance because they correctly optimize for time-average growth, not ensemble expectations.

## Next Steps

- **Scale up simulations:** [core/04_monte_carlo_simulation.ipynb](04_monte_carlo_simulation.ipynb)
- **Quantify tail risk:** [core/05_risk_metrics.ipynb](05_risk_metrics.ipynb)
- **Analyze long-term dynamics:** [core/06_long_term_dynamics.ipynb](06_long_term_dynamics.ipynb)
- **Understand growth drivers:** [core/07_growth_dynamics.ipynb](07_growth_dynamics.ipynb)